<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [4]:
import numpy as np
import numpy.linalg as lin

In [5]:
def similarity(v1, v2):
    n1 = lin.norm(v1)
    n2 = lin.norm(v2)
    if n1 == 0 or n2 == 0:
        return 0
    return (v1 @ v2) / (n1 * n2)

sim_dog = np.zeros(len(wv_embeddings))
i = 0
for w in wv_embeddings:
    sim_dog[i] = similarity(wv_embeddings["dog"], w)
    i += 1
    if i == len(wv_embeddings):
        break

In [6]:
max_indices = np.argsort(sim_dog)[-6:-1]
for w in max_indices[::-1]:
    print(wv_embeddings.index_to_key[w], sim_dog[w])

animal 0.8564179539680481
dogs 0.7880865931510925
mammal 0.7623804807662964
cats 0.7621252536773682
animals 0.760793924331665


Как и ожидалось, кошка похожа на собаку

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [207]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    question_emb = np.zeros(dim)
    used_words = 0
    for w in tokenizer.tokenize(question):
        if w in embeddings:
            question_emb += embeddings[w]
            used_words += 1
        
    if used_words == 0:
        return question_emb
    
    return question_emb / used_words    #variant 1
    # return question_emb               #variant 2

In [9]:
q = "hello, I am a human"
tokenizer = WordPunctTokenizer()
question_to_vec(q, wv_embeddings, tokenizer)
;

''

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [10]:
question_to_vec("I love neural networks", wv_embeddings, tokenizer)

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114,
        0.28913016,  1.03125   , -2.86546615, -1.31928726,  2.20982948,
        1.16786722,  1.08419518,  1.84672268, -0.98733809,  0.87209455,
       -0.17219625,  0.73839017, -0.78278264, -0.88278695, -0.61526108,
       -0.15872356, -0.61707981,  0.36668165,  2.01375335,  0.26274722,
       -1.4950124 , -1.38671508, -1.20499369, -1.01661696,  2.76211715,
        1.72557124, -0.19038178, -0.58994474, -0.1141155 , -1.88903529,
       -1.78835048, -0.20420383, -0.70620341,  2.4890339 , -0.15452717,
        1.12550129,  0.26108352,  1.31532462, -0.57667526, -0.33271411,
        1.67649019,  0.91229995, -0.6988014 ,  2.37525521, -1.76342229,
        3.13780514,  1.22432033, -1.05544141,  0.78975705,  0.04404024,
        1.16230452, -0.44780231, -0.9149816 ,  0.01501505, -0.85523087,
       -1.01834532, -0.71621163,  1.0727205 ,  0.88818762, -0.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$\text{Hits(K)} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K]$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Если вопрос состоит в том, когда достигается максимум, то понятно, что тогда, когда ранг дубликатов > 1 и <= 47

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits(1)} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits(4)} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG(1)} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DC(4)} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

- [K = 10] $\text{DCG(10)} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [11]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    for r in dup_ranks:
        if r <= k:
            hits_value += 1
            
    hits_value /= len(dup_ranks)
    return hits_value

In [12]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    for r in dup_ranks:
        if r <= k:
            dcg_value += 1 / np.log2(1 + r)

    dcg_value /= len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

Видимо подразумевается, что мы засунем в candidates_ranking дубликат

In [13]:
import pandas as pd

In [14]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu",
                       "How does the caught keyword determine type of exception"],]

copy_pos = [4,]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [0] * len(copy_answers)

In [15]:
q_emb = question_to_vec(copy_answers[0], wv_embeddings, tokenizer)

sim_q = np.array([similarity(q_emb, question_to_vec(q, wv_embeddings, tokenizer)) for q in candidates_ranking[0]])

ranked_candidates = np.argsort(sim_q)[::-1]

rank = 0
for i in ranked_candidates:
    rank += 1
    if i == copy_pos[0]:
        dup_ranks[0] = rank

In [16]:
ranked_candidates

array([1, 4, 2, 0, 3], dtype=int64)

In [17]:
# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [18]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [112]:
!unzip stackoverflow_similar_questions.zip

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Считайте данные.

In [154]:
def read_corpus(filename):
    data = []
    i = 0
    for line in open(filename, encoding='utf-8'):
        parsed_line = line.split(sep='\t')
        data.append(parsed_line)
        i += 1
        if i % 100 == 0:
            print("Processed: ", i)
    print(len(data), len(data[0]))
    return data

validation_data = read_corpus('./data/validation.tsv')

Processed:  100
Processed:  200
Processed:  300
Processed:  400
Processed:  500
Processed:  600
Processed:  700
Processed:  800
Processed:  900
Processed:  1000
Processed:  1100
Processed:  1200
Processed:  1300
Processed:  1400
Processed:  1500
Processed:  1600
Processed:  1700
Processed:  1800
Processed:  1900
Processed:  2000
Processed:  2100
Processed:  2200
Processed:  2300
Processed:  2400
Processed:  2500
Processed:  2600
Processed:  2700
Processed:  2800
Processed:  2900
Processed:  3000
Processed:  3100
Processed:  3200
Processed:  3300
Processed:  3400
Processed:  3500
Processed:  3600
Processed:  3700
3760 1001


In [142]:
validation_data[-1][288]

'is table.re'

Кол-во строк

In [143]:
len(validation_data)

3760

Размер нескольких первых строк

In [144]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [145]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [202]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    q_emb = question_to_vec(question, embeddings, tokenizer, dim=dim)

    sim_q = np.array([similarity(q_emb, question_to_vec(q, embeddings, tokenizer, dim=dim)) for q in candidates])
    
    ranked_candidates = np.argsort(sim_q)[::-1]
    
    res = [(ranked_candidates[i], candidates[ranked_candidates[i]]) for i in range(len(sim_q))]
    
    return res

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [147]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [149]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]


Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [150]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'WPF- How to update the changes in list item of a list'), 
            (0, 'Getting all list items of an unordered list in PHP'), 
            (2, 'select2 not displaying search results')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Тоже 102 :0

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [151]:
from tqdm.notebook import tqdm

In [152]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [153]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.226 | Hits@   1: 0.226
DCG@   5: 0.282 | Hits@   5: 0.335
DCG@  10: 0.298 | Hits@  10: 0.387
DCG@ 100: 0.345 | Hits@ 100: 0.621
DCG@ 500: 0.371 | Hits@ 500: 0.824
DCG@1000: 0.389 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [155]:
train_data = read_corpus('./data/train.tsv')

Processed:  100
Processed:  200
Processed:  300
Processed:  400
Processed:  500
Processed:  600
Processed:  700
Processed:  800
Processed:  900
Processed:  1000
Processed:  1100
Processed:  1200
Processed:  1300
Processed:  1400
Processed:  1500
Processed:  1600
Processed:  1700
Processed:  1800
Processed:  1900
Processed:  2000
Processed:  2100
Processed:  2200
Processed:  2300
Processed:  2400
Processed:  2500
Processed:  2600
Processed:  2700
Processed:  2800
Processed:  2900
Processed:  3000
Processed:  3100
Processed:  3200
Processed:  3300
Processed:  3400
Processed:  3500
Processed:  3600
Processed:  3700
Processed:  3800
Processed:  3900
Processed:  4000
Processed:  4100
Processed:  4200
Processed:  4300
Processed:  4400
Processed:  4500
Processed:  4600
Processed:  4700
Processed:  4800
Processed:  4900
Processed:  5000
Processed:  5100
Processed:  5200
Processed:  5300
Processed:  5400
Processed:  5500
Processed:  5600
Processed:  5700
Processed:  5800
Processed:  5900
Proces

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [158]:
import spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 3.2 MB/s eta 0:00:04
     ---------------------------------------- 0.1/12.8 MB 1.7 MB/s eta 0:00:08
      --------------------------------------- 0.3/12.8 MB 2.1 MB/s eta 0:00:06
     - -------------------------------------- 0.5/12.8 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.1/12.8 MB 4.8 MB/s eta 0:00:03
     ------ --------------------------------- 2.0/12.8 MB 7.6 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 10.5 MB/s eta 0:00:01
     ---------------- ----------------------- 5.4/12.8 MB 14.8 MB/s eta 0:00:01
     ----------------------- ---------------- 7.7/12.8 MB 18.8 MB/s eta 0:00:01
     ----------------------- ---------------- 7.7/12.8 MB 18.8 MB/s eta 0:00:01
     ----------------------------- ---------- 9.6/12.8 MB 19.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8

In [160]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

def preproc_spacy(text):
    spacy_results = nlp(text)
    return ' '.join([token.lemma_ for token in spacy_results if token.lemma_ not in stopWords])

In [168]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

def preproc_nltk(text):
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\koman\AppData\Roaming\nltk_data...


In [169]:
words = [[preproc_nltk(text).split() for text in line] for line in train_data]

In [179]:
words_flat = [w for line in words for w in line]

Потестируем разные параметры Word2Vec

In [203]:
from gensim.models import Word2Vec
def test_embeddings(data, embeddings, tokenizer, max_validation_examples=1000, dim=200):
    wv_ranking = []
    for i, line in enumerate(tqdm(data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings, tokenizer, dim=dim)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [190]:
embeddings_trained1 = Word2Vec(words_flat,  # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=7,  # consider words that occurred at least 5 times
                              window=5).wv

In [191]:
test_embeddings(validation_data, embeddings_trained1, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.174 | Hits@   1: 0.174
DCG@   5: 0.229 | Hits@   5: 0.279
DCG@  10: 0.246 | Hits@  10: 0.331
DCG@ 100: 0.292 | Hits@ 100: 0.555
DCG@ 500: 0.322 | Hits@ 500: 0.796
DCG@1000: 0.344 | Hits@1000: 1.000


In [192]:
embeddings_trained2 = Word2Vec(words_flat,  # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=3,  # consider words that occurred at least 5 times
                              window=3).wv

In [193]:
test_embeddings(validation_data, embeddings_trained2, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.172 | Hits@   1: 0.172
DCG@   5: 0.227 | Hits@   5: 0.276
DCG@  10: 0.244 | Hits@  10: 0.330
DCG@ 100: 0.288 | Hits@ 100: 0.547
DCG@ 500: 0.319 | Hits@ 500: 0.790
DCG@1000: 0.341 | Hits@1000: 1.000


In [196]:
embeddings_trained3 = Word2Vec(words_flat,  # data for model to train on
                              vector_size=400,  # embedding vector size
                              min_count=7,  # consider words that occurred at least 5 times
                              window=5).wv

In [199]:
test_embeddings(validation_data, embeddings_trained3, tokenizer, dim=400)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.170 | Hits@   1: 0.170
DCG@   5: 0.229 | Hits@   5: 0.282
DCG@  10: 0.248 | Hits@  10: 0.343
DCG@ 100: 0.292 | Hits@ 100: 0.562
DCG@ 500: 0.321 | Hits@ 500: 0.791
DCG@1000: 0.343 | Hits@1000: 1.000


Интересно, что с большим vec_size качество получилось ниже. Но вообще говоря, колебания качества незначительны.

А что будет, если при подсчете эмбеддинга предложения брать сумму, а не среднее?

In [209]:
test_embeddings(validation_data, embeddings_trained1, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.174 | Hits@   1: 0.174
DCG@   5: 0.229 | Hits@   5: 0.279
DCG@  10: 0.246 | Hits@  10: 0.331
DCG@ 100: 0.292 | Hits@ 100: 0.555
DCG@ 500: 0.322 | Hits@ 500: 0.796
DCG@1000: 0.344 | Hits@1000: 1.000


Удивительно, но результат точь-в-точь такой же (хотя нет, не удивительно). В целом, понятно почему: ведь длина вектора нас не интересует, нас интересует его направление. Так что можно не брать среднее, а, например, нормировать вектор, чтобы у всех эмбеддингов норма была 1. Тогда и косинус угла можно считать без деления на норму.

Попробуем запустить Word2Vec без лемматизации.

In [211]:
def preproc_nltk_without_lemmatize(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])

In [212]:
words_without_lemmatize = [[preproc_nltk_without_lemmatize(text).split() for text in line] for line in train_data]
words_flat_without_lemmatize = [w for line in words for w in line]

In [213]:
embeddings_trained4 = Word2Vec(words_flat_without_lemmatize,  # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=7,  # consider words that occurred at least 5 times
                              window=5).wv

In [214]:
test_embeddings(validation_data, embeddings_trained4, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.176 | Hits@   1: 0.176
DCG@   5: 0.230 | Hits@   5: 0.279
DCG@  10: 0.251 | Hits@  10: 0.344
DCG@ 100: 0.294 | Hits@ 100: 0.558
DCG@ 500: 0.324 | Hits@ 500: 0.788
DCG@1000: 0.346 | Hits@1000: 1.000


А вот это уже интересно, почему-то лемматизация только ухудшило качество.

Попробуем увеличить min_count

In [215]:
embeddings_trained5 = Word2Vec(words_flat_without_lemmatize,  # data for model to train on
                               vector_size=200,  # embedding vector size
                               min_count=15,  # consider words that occurred at least 5 times
                               window=5).wv
test_embeddings(validation_data, embeddings_trained5, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.173 | Hits@   1: 0.173
DCG@   5: 0.229 | Hits@   5: 0.280
DCG@  10: 0.247 | Hits@  10: 0.334
DCG@ 100: 0.292 | Hits@ 100: 0.556
DCG@ 500: 0.322 | Hits@ 500: 0.788
DCG@1000: 0.344 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
1. Простого применения регулярных выражений может некорректно разбить предложение на слова, так как мало того, что могут быть сложные конструкции по типу "Mr. Watson", которое трудно разбить на одно слово, хотя по смыслу оно им и является, так еще и в предложениях могут быть ошибки, так что регулярки точно не самая хорошая идея. 
    Понятно, что для конкретно нашей задачи, где весь разделяемый текст это одно правильно
составленное предложение, где вряд ли есть много сложных грамматических конструкций, вполне можно использовать относительные дешевые регулярки, но в более общем случае нужно трижды подумать, прежде чем их использовать.
    Я использовал work_tokenize, который использует TreebankWordTokenizer. Он вначале некоторым
образом видоизменяет текст, удаляя лишние и производя разделение на слова и знаки препинания, затем некоторым хитрым образом делит на слова с учетом контекста. Это наш слон.
2. Интересно, что лемматизация только ухудшило качество. У меня есть 2 гипотезы:
    1. До лемматизации было много похожих слов, но немного разных, при этом по количеству их было мало и они отбрасывались. После лемматизации такие слова слились в одно с большим количеством повторений и его уже не отбросили, но все равно количество было слишком маленьким, так что качество от этого только ухудшилось. Нужно попробовать увеличить порог min_count
    2. Лемматизация исказила смысл слов и они потеряли вложенную в них информацию. Такое могло произойти из-за некачественной лемматизации, либо из-за специфики текста, так как он нагружен различными терминами, которые нельзя изменять.

    После увеличения min_count качество только упало, так что скорее всего имеет место быть вторая гипотеза.
3. Лучше работают скаченные эмбеддинги. Думаю такое происходит из-за малого объема наших данных, из-за маленьких предложений в этих данных, из-за специфичной лексики и смешения вопросительных и утвердительных предложений. Понятно, что данные со StackOverflow имеют немного другую структуру и более репрезентативные. Как минимум из-за того, что данных намного больше и предложения в среднем длиннее. Также понятно, что идеологически наши данные являются подмножеством данных со StackOverflow, так как и там и там рассматриваются какие-то технические вопросы из смежных тем.
4. Качество не самое лучшее из-за глупого представления предложения. Понятно, что сумма эмбеддингов слов это крайне плохой эмбеддинг предложения, так как не учитывается взаимосвязь слов, их положение.
5. Нужно придумать задачу, чтобы при ее решении можно было получить нормальные эмбеддинги предложений. Понятно, что перед этим нужно обработать предложения с использованием лемматизации и других видов нормировки. Еще хочется новую идею, не word2vec, так как, как мне кажется, далеко не всегда стоит использовать слова из разных предложений, так как они далеко не всегда связаны логически (например, как в нашем датасете).